# Validation Type 1

VALIDATION CLASS 0: collect the fake images already generated

In [1]:
import pandas as pd

df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/validation_synthetics.csv')

# Filter rows where '6_way_label' equals 0 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows_SD = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('SD'))]
filtered_rows_DL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('DL'))]
filtered_rows_GL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('GL'))]

# Concatenate columns for all groups
result = pd.concat([filtered_rows_SD, filtered_rows_DL, filtered_rows_GL])
print(len(result))

result.to_csv('C:/Users/nello/Documents/vscode_projects/Thesis/5_biased_detection/csv/validation/validation_class_0_already_generated.csv', index=False)

5602


save a csv for the 11.490 pristine and the 11.490 - 5.602 to be generated

In [25]:
import pandas as pd

MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_0 = "csv/validation/validation_class_0_already_generated.csv"

validation = pd.read_csv(MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV, sep='\t')
validation_filtered = validation[validation['6_way_label'] == 0]

validation_class_0_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_0)
print("len of validation_class_0_already_generated: "+str(len(validation_class_0_already_generated)))

# Filter rows from train DataFrame that are not in the 'id' column of the other DataFrame
validation_not_in_other = validation_filtered[~validation_filtered['id'].isin(validation_class_0_already_generated['id'])]

# Select the first 11,490 rows from train_not_in_other
validation_pristine = validation_not_in_other.head(11491)

# Save train_pristine to CSV
print("len of validation_class_0_pristine: "+str(len(validation_pristine)))
validation_pristine.to_csv('csv/validation/validation_class_0_pristine.csv', index=False)

# Filter rows from train_filtered that are not in train_class_0_already_generated or train_pristine
validation_remaining = validation_filtered[~validation_filtered['id'].isin(validation_class_0_already_generated['id']) & ~validation_filtered['id'].isin(validation_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of validation_class_0_to_be_generated: "+str(len(validation_remaining)))
validation_remaining.to_csv('csv/validation/validation_class_0_to_be_generated.csv', index=False)
print("validation_class_0_already_generated: "+str(len(validation_class_0_already_generated))+" + validation_class_0_to_be_generated: "+str(len(validation_remaining))+" = " + str(len(validation_class_0_already_generated)+len(validation_remaining)))

len of validation_class_0_already_generated: 5602
len of validation_class_0_pristine: 11491
len of validation_class_0_to_be_generated: 5888
validation_class_0_already_generated: 5602 + validation_class_0_to_be_generated: 5888 = 11490


rename the columns real_image and fake_image to pristine_image and generated_image inside validation_class_0_already_generated.csv

In [26]:
df = pd.read_csv('csv/validation/validation_class_0_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/validation/validation_class_0_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in validation_class_0_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [33]:
df = pd.read_csv('csv/validation/validation_class_0_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/validation/validation_class_0_already_generated.csv", index=False)

reduce columns of validation_class_0_to_be_generated and validation_class_0_pristine csv:

In [34]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_0_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_0_to_be_generated.csv", index=False)

In [35]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_0_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_0_pristine.csv", index=False)

check if every id inside the validation_tsv_with_class.csv is contained in the union of the 3 csv validation_pristine + validation_class_0_already_generated + validation_class_0_to_be_generated

In [30]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/validation/validation_class_0_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/validation/validation_class_0_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/validation/validation_class_0_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv", sep='\t')['6_way_label'] == 0, 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 0.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 0.")

22981
22981
The union of IDs from the first three CSVs corresponds exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 0.


Now i print out how many StableDiffusion, Dreamlike and Glide images i have already generated:

In [31]:
df = pd.read_csv("csv/validation/validation_class_0_already_generated.csv")

# Count the rows where the 'fake_id' column starts with 'SD', 'DL', or 'GL'
count_SD = len(df[df['fake_id'].str.startswith('SD')])
count_DL = len(df[df['fake_id'].str.startswith('DL')])
count_GL = len(df[df['fake_id'].str.startswith('GL')])

print("Number of rows where 'fake_id' starts with 'SD':", count_SD)
print("Number of rows where 'fake_id' starts with 'DL':", count_DL)
print("Number of rows where 'fake_id' starts with 'GL':", count_GL)

Number of rows where 'fake_id' starts with 'SD': 2263
Number of rows where 'fake_id' starts with 'DL': 2242
Number of rows where 'fake_id' starts with 'GL': 1097


Thus, to get to 11.490 generated images i want to generate other:

- SD: 4.000 - 2.263 = 1.737
- DL: 4.000 - 2.242 = 1.758
- GL: (11.490 - 8.000) - 1.097 = 2.393

so 5.888 in total from the validation_class_0_to_be_generated.csv, but first i need to caption all the images from validation_class_0_to_be_generated.csv

Here we check that every pristine image inside validation_class_0_to_be_generated csv is already in the image folder

In [36]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_0_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 5888


Here we check that every pristine image inside validation_class_0_pristine csv is already in the image folder

In [37]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_0_pristine.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 11491


Here we check that every pristine image inside validation_class_0_already_generated csv is already in the image folder

In [38]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_0_already_generated.csv")['fake_id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 5602


-------------------------

# Validation Type 2

Now construct the validation "type 2" part, thus the 8.963 pristine with 6_way_label == 1,2,3,5 (4 manipulated deleted) and 8.963 generated still with 6_way_label == 1,2,3,5

In [7]:
import pandas as pd

df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/validation_synthetics.csv')

# Filter rows where '6_way_label' equals 1, 2, 3, or 5 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows = df[(df['6_way_label'].isin([1, 2, 3, 5])) & (df['fake_id'].str.startswith(('SD', 'DL', 'GL')))]

# Save the result to a new CSV file
print(len(filtered_rows))
filtered_rows.to_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv', index=False)

4398


save a csv for the 8.963 pristine and the 8.963-4.398 to be generated

In [39]:
import pandas as pd

MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_1_2_3_5 = "csv/validation/validation_class_1_2_3_5_already_generated.csv"

validation = pd.read_csv(MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV, sep='\t')
print("validation len: "+str(len(validation)))
validation_filtered = validation[validation['6_way_label'].isin([1, 2, 3, 5])]

validation_class_1_2_3_5_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_1_2_3_5)
print("len of validation_class_1_2_3_5_already_generated: "+str(len(validation_class_1_2_3_5_already_generated)))

# Filter rows from val DataFrame that are not in the 'id' column of the other DataFrame
validation_not_in_other = validation_filtered[~validation_filtered['id'].isin(validation_class_1_2_3_5_already_generated['id'])]

# Select the first 8.963 rows from validation_not_in_other
validation_pristine = validation_not_in_other.head(8964)

# Save validation_pristine to CSV
print("len of validation_class_1_2_3_5_pristine: "+str(len(validation_pristine)))
validation_pristine.to_csv('csv/validation/validation_class_1_2_3_5_pristine.csv', index=False)

# Filter rows from validation_filtered that are not in validation_class_1_2_3_5_already_generated or validation_pristine
validation_remaining = validation_filtered[~validation_filtered['id'].isin(validation_class_1_2_3_5_already_generated['id']) & ~validation_filtered['id'].isin(validation_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of validation_class_1_2_3_5_to_be_generated: "+str(len(validation_remaining)))
validation_remaining.to_csv('csv/validation/validation_class_1_2_3_5_to_be_generated.csv', index=False)
print("validation_class_1_2_3_5_already_generated: "+str(len(validation_class_1_2_3_5_already_generated))+"  + validation_class_1_2_3_5_to_be_generated: "+str(len(validation_remaining))+" = " + str(len(validation_class_1_2_3_5_already_generated)+len(validation_remaining)))

validation len: 57489
len of validation_class_1_2_3_5_already_generated: 4398
len of validation_class_1_2_3_5_pristine: 8964
len of validation_class_1_2_3_5_to_be_generated: 4566
validation_class_1_2_3_5_already_generated: 4398  + validation_class_1_2_3_5_to_be_generated: 4566 = 8964


rename the columns real_image and fake_image to pristine_image and generated_image inside validation_class_1_2_3_5_already_generated.csv

In [40]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in validation_class_1_2_3_5_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [41]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv", index=False)

reduce columns of validation_class_1_2_3_5_to_be_generated and validation_class_1_2_3_5_pristine csv:

In [42]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 0
df['fakenews_text'] = 1

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_1_2_3_5_to_be_generated.csv", index=False)

In [43]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 0
df['fakenews_text'] = 1

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_1_2_3_5_pristine.csv", index=False)

Show the distribution of the original classes (1,2,3,5) for the csv

In [44]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_pristine.csv')

# Count the number of rows for each value of '6_way_label'
label_counts = df['6_way_label'].value_counts()

# Print the counts for each unique value of '6_way_label'
print("Counts for each unique value of '6_way_label':")
print(label_counts)

Counts for each unique value of '6_way_label':
6_way_label
2    5570
1    1701
5    1082
3     611
Name: count, dtype: int64


In [45]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Count the number of rows for each value of '6_way_label'
label_counts = df['6_way_label'].value_counts()

# Print the counts for each unique value of '6_way_label'
print("Counts for each unique value of '6_way_label':")
print(label_counts)

Counts for each unique value of '6_way_label':
6_way_label
2    2696
1     890
5     520
3     292
Name: count, dtype: int64


In [46]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Group the DataFrame by '6_way_label'
grouped = df.groupby('6_way_label')

# Iterate over each group
for label, group in grouped:
    # Filter the group where 'id' column starts with "SD", "GL", or "DL"
    sd_count = len(group[group['fake_id'].str.startswith('SD')])
    gl_count = len(group[group['fake_id'].str.startswith('GL')])
    dl_count = len(group[group['fake_id'].str.startswith('DL')])
    
    # Print counts for each value of '6_way_label'
    print(f"Counts for '6_way_label' {label}:")
    print(f"Starts with 'SD': {sd_count}")
    print(f"Starts with 'GL': {gl_count}")
    print(f"Starts with 'DL': {dl_count}")
    print()


Counts for '6_way_label' 1:
Starts with 'SD': 348
Starts with 'GL': 186
Starts with 'DL': 356

Counts for '6_way_label' 2:
Starts with 'SD': 1065
Starts with 'GL': 552
Starts with 'DL': 1079

Counts for '6_way_label' 3:
Starts with 'SD': 119
Starts with 'GL': 50
Starts with 'DL': 123

Counts for '6_way_label' 5:
Starts with 'SD': 205
Starts with 'GL': 115
Starts with 'DL': 200



In [13]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_to_be_generated.csv')

# Define the start and end indices for each interval
intervals = {
    "Generated with SD": (0, 12350),
    "Generated with DL": (12350, 24394),
    "Generated with GL": (24394, len(df))
}

# Iterate over each interval
for method, (start_index, end_index) in intervals.items():
    # Filter the DataFrame based on the interval
    interval_df = df.iloc[start_index:end_index]
    
    # Group the interval DataFrame by '6_way_label'
    grouped = interval_df.groupby('6_way_label')
    
    # Print the count of rows with each '6_way_label' value within the interval
    print(f"{method}:")
    for label, group in grouped:
        label_count = len(group)
        print(f"6_way_label: {label} in the interval [{start_index}, {end_index}): {label_count}")
    print()

Generated with SD:
6_way_label: 1 in the interval [0, 12350): 2356
6_way_label: 2 in the interval [0, 12350): 7614
6_way_label: 3 in the interval [0, 12350): 860
6_way_label: 5 in the interval [0, 12350): 1520

Generated with DL:
6_way_label: 1 in the interval [12350, 24394): 2315
6_way_label: 2 in the interval [12350, 24394): 7437
6_way_label: 3 in the interval [12350, 24394): 806
6_way_label: 5 in the interval [12350, 24394): 1486

Generated with GL:
6_way_label: 1 in the interval [24394, 40997): 3146
6_way_label: 2 in the interval [24394, 40997): 10315
6_way_label: 3 in the interval [24394, 40997): 1080
6_way_label: 5 in the interval [24394, 40997): 2062



check if every id inside the train_tsv_with_class.csv with '6_way_label' = 1,2,3,5 is contained in the union of the 3 csv train_class_1_2_3_5_pristine + train_class_1_2_3_5_already_generated + train_class_1_2_3_5_to_be_generated

In [12]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t')['6_way_label'].isin([1, 2, 3, 5]), 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.")

170774
170774
The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.


Thus, to get to 85.387 generated images i want to generate other 40.997:

- SD: 30.000 - 17.650 = 12.350
- DL: 30.000 - 17.956 = 12.044
- GL: (85.387 - 60.000) - 8.784 = 16.603

so 40.997 in total from the train_class_1_2_3_5_to_be_generated.csv, but first i need to caption all the images from train_class_1_2_3_5_to_be_generated.csv

Here we check that every pristine image inside train_class_1_2_3_5_to_be_generated csv is already in the image folder

In [2]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "C:/Users/nello/Desktop/TESI/dataset_after_merging_WITH_DUPLICATES"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 40997


Here we check that every pristine image inside train_class_1_2_3_5_pristine csv is already in the image folder

In [18]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_pristine.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 85387


Here we check that every pristine image inside train_class_1_2_3_5_already_generated csv is already in the image folder

In [17]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/train/train_class_1_2_3_5_already_generated.csv")['fake_id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 44390


Clean the dataset folder from Manipulated Images (class 4)

In [4]:
import os
import pandas as pd

csv_files = ['/home/enriconello/DeepFakeDetection/Thesis/3_image_generation/generated_images/csv/train.csv',
             '/home/enriconello/DeepFakeDetection/Thesis/3_image_generation/generated_images/csv/test_with_duplicates.csv',
             '/home/enriconello/DeepFakeDetection/Thesis/3_image_generation/generated_images/csv/validation_with_duplicates.csv']

# image folder path 
image_folder = '/home/enriconello/DeepFakeDetection/dataset'

unique_ids = set()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    
    # Filter rows where '6_way_label' equals to the class Manipulated images
    filtered_df = df[df['6_way_label'] == 4]
    unique_ids.update(filtered_df['id'])

num_images_removed = 0

for id_value in unique_ids:
    image_path = os.path.join(image_folder, f"{id_value}.jpg")
    
    if os.path.exists(image_path):
        os.remove(image_path)
        num_images_removed += 1
    else:
        continue

print(f"Total number of images removed: {num_images_removed}")

Total number of images removed: 190185


Clean the train_class_1_2_3_5_captioned csv, deleting the generated captions that starts with "araf" (buggy captions)

In [9]:
import pandas as pd

original_df = pd.read_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_captioned.csv')

# Create a copy of the original DataFrame
modified_df = original_df.copy()

# Remove words starting with "araf" in the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.replace(r'\baraf\w*\b', '', regex=True)

# Remove leading and trailing whitespace from the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.strip()

# Save the modified DataFrame to the same CSV file
modified_df.to_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_captioned.csv", index=False)

# Count the number of updated rows
updated_rows_count = (original_df['generated_caption'] != modified_df['generated_caption']).sum()

print(f"Number of updated rows: {updated_rows_count}")

Number of updated rows: 14525


------------------

# After fake generation

After generating all the fakes images that were missing, i need to merge the class_0_to_be_generated and the class_0_already_generated, and the same for class 1,2,3,5

Starting by adding the "fake_id" column to the train_class_0_to_be_generated csv

In [7]:
import pandas as pd

df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_0_to_be_generated.csv")

# new column 'fake_id'
df['fake_id'] = ''

# assign values to 'fake_id' based on row index intervals
df.loc[df.index < 17650, 'fake_id'] = 'SD_fake_' + df['id']
df.loc[(df.index >= 17650) & (df.index < 35606), 'fake_id'] = 'DL_fake_' + df['id']
df.loc[df.index >= 35606, 'fake_id'] = 'GL_fake_' + df['id']

# 'fake_id' column next to 'id' column
df.insert(df.columns.get_loc('id') + 1, 'fake_id', df.pop('fake_id'))

df.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_0_final.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_to_be_generated.csv")

# new column 'fake_id'
df['fake_id'] = ''

# assign values to 'fake_id' based on row index intervals
df.loc[df.index < 12350, 'fake_id'] = 'SD_fake_' + df['id']
df.loc[(df.index >= 12350) & (df.index < 24394), 'fake_id'] = 'DL_fake_' + df['id']
df.loc[df.index >= 24394, 'fake_id'] = 'GL_fake_' + df['id']

# 'fake_id' column next to 'id' column
df.insert(df.columns.get_loc('id') + 1, 'fake_id', df.pop('fake_id'))

df.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/train/train_class_1_2_3_5_final.csv', index=False)

set the column pristine_image = 0 and generated_image = 1 for every row in train_class_0_final csv

In [8]:
df = pd.read_csv('csv/train/train_class_0_final.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train/train_class_0_final.csv", index=False)

In [ ]:
df = pd.read_csv('csv/train/train_class_1_2_3_5_final.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/train/train_class_1_2_3_5_final.csv", index=False)

Now i need to add the column "generated_caption"

In [25]:
import pandas as pd

df_a = pd.read_csv("csv/train/train_class_0_final.csv")
df_b = pd.read_csv("csv/train/train_class_0_captioned.csv")

# Perform inner join on 'id' column
df_c = pd.merge(df_a, df_b, on='id', how='inner')

# Rename 'original_caption_x' to 'original_caption'
df_c = df_c.rename(columns={'original_caption_x': 'original_caption'})

df_c = df_c[['id', 'fake_id', 'author', 'original_caption', 'generated_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']]

df_c.to_csv("csv/train/train_class_0_final.csv", index=False)
df_c.head()

,id,fake_id,author,original_caption,generated_caption,num_comments,6_way_label,pristine_image,generated_image,real_text,fakenews_text
0,dhr269,SD_fake_dhr269,bico89,my bill was,someone is holding a receipt and a box of cheese,10.0,0,0,1,1,0
1,3gb4sc,SD_fake_3gb4sc,landofthemolotovia,quebec considering removing nword from place n...,there is a bear that is walking across the water,19.0,0,0,1,1,0
2,1wkhcb,SD_fake_1wkhcb,Zjw0115,dramatic baseball catcher,baseball player catching a ball in a stadium,6.0,0,0,1,1,0
3,bt0p1w,SD_fake_bt0p1w,tx69er,this mcdonalds uses aol for its corporate email,a close up of a receipt with a price tag on it,3.0,0,0,1,1,0
4,2yfbtu,SD_fake_2yfbtu,ProfessorFartdust,fan and katy perry,woman with a pink and blue hair and a man with...,5.0,0,0,1,1,0


In [ ]:
import pandas as pd

df_a = pd.read_csv("csv/train/train_class_1_2_3_5_final.csv")
df_b = pd.read_csv("csv/train/train_class_1_2_3_5_captioned.csv")

# Perform inner join on 'id' column
df_c = pd.merge(df_a, df_b, on='id', how='inner')

# Rename 'original_caption_x' to 'original_caption'
df_c = df_c.rename(columns={'original_caption_x': 'original_caption'})

df_c = df_c[['id', 'fake_id', 'author', 'original_caption', 'generated_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']]

df_c.to_csv("csv/train/train_class_1_2_3_5_final2.csv", index=False)
df_c.head()

finally merge train_class_0_final and train_class_1_2_3_5_final to create train_final csv

In [ ]:
import pandas as pd

# Read CSV files
df1 = pd.read_csv("csv/train/train_class_0_final.csv")
df2 = pd.read_csv("csv/train/train_class_1_2_3_5_final.csv")

# Concatenate DataFrames
df_union = pd.concat([df1, df2], ignore_index=True)

# Save the result to CSV file
df_union.to_csv("csv/train/train_final.csv", index=False)
print(len(df_union))